# Direct Deep Reinforcement Learning

## A. Negociação por Reforço Direto (Direct Reinforcement Trading)

Nesta seção, os autores revisam a estrutura de *aprendizado por reforço direto* (DRL - *Direct Reinforcement Learning*), originalmente proposta por {cite:p}`Moody98`. Eles mostram que esse tipo de DRL pode ser interpretado como uma **rede neural recorrente (RNN)** com apenas **uma camada**, o que a torna um modelo relativamente simples, mas ainda assim útil.

O ambiente é modelado como uma sequência de preços de um ativo financeiro ao longo do tempo:

$$
p_1, p_2, \dots, p_t, \dots
$$

A variação de preço (ou retorno) entre dois instantes consecutivos é definida por:

$$
z_t = p_t - p_{t-1}
$$

Com base nas condições atuais do mercado, o modelo toma uma **decisão de negociação em tempo real** a cada instante de tempo $t$. Essa decisão, chamada **$\delta_t$**, pode assumir os seguintes valores:

- $1$: posição comprada (*long*)
- $0$: posição neutra (*neutral*)
- $-1$: posição vendida (*short*)

O **lucro (ou prejuízo) no tempo $t$**, representado por $R_t$, é dado pela seguinte equação:

$$
R_t = \delta_{t-1} \cdot z_t - c \cdot \left| \delta_t - \delta_{t-1} \right|
\tag{1}
$$

### Interpretação da fórmula:

- O primeiro termo $\delta_{t-1} \cdot z_t$ representa o ganho (ou perda) obtido com a variação do mercado, **de acordo com a posição assumida no período anterior**.
  - Exemplo: se você estava comprado ($\delta_{t-1} = 1$) e o preço subiu ($z_t > 0$), você tem lucro.
  - Se estava vendido ($\delta_{t-1} = -1$) e o preço caiu ($z_t < 0$), também há lucro.
  - Se estava neutro ($\delta_{t-1} = 0$), esse termo é nulo.

- O segundo termo $c \cdot \left| \delta_t - \delta_{t-1} \right|$ representa o **custo de transação (transaction cost - TC)**. Ele é cobrado **apenas quando há mudança de posição** entre dois instantes consecutivos.
  - Exemplo: mudar de neutro para comprado gera custo.
  - Manter a mesma posição ($\delta_t = \delta_{t-1}$) **não gera custo**.

O parâmetro $c$ representa esse custo, que pode incluir taxas da corretora, impostos, ou perdas por slippage (diferença entre o preço esperado e o executado). Esses custos são importantes para simular de forma realista os impactos de decisões frequentes de compra e venda no lucro final.




## Exemplos de Ganho e Perda com a Fórmula de Reforço Direto

A fórmula do lucro/prejuízo a cada instante de tempo $ t $ é:

$$
R_t = \delta_{t-1} \cdot z_t - c \cdot |\delta_t - \delta_{t-1}|
$$

Onde:

- $ \delta_t \in \{-1, 0, 1\} $: posição assumida (vendida, neutra ou comprada)
- $ z_t = p_t - p_{t-1} $: variação de preço no período
- $ c $: custo de transação (por exemplo, $ c = 0.5 $)

---

### Exemplo 1: Lucro com posição comprada

- $ p_{t-1} = 100 $
- $ p_t = 103 $ → $ z_t = 3 $
- $ \delta_{t-1} = 1 $
- $ \delta_t = 1 $
- $ c = 0.5 $

**Cálculo:**

$$
R_t = 1 \cdot 3 - 0.5 \cdot |1 - 1| = 3 - 0 = \boxed{3}
$$

**Lucro de 3 unidades**.

---

### Exemplo 2: Prejuízo com posição comprada

- $ z_t = -2 $ (queda de preço)
- $ \delta_{t-1} = 1 $
- $ \delta_t = 1 $
- $ c = 0.5 $

**Cálculo:**

$$
R_t = 1 \cdot (-2) - 0.5 \cdot |1 - 1| = -2 - 0 = \boxed{-2}
$$

**Prejuízo de 2 unidades**, sem custo de transação.

---

### Exemplo 3: Lucro com posição vendida

- $ z_t = -4 $
- $ \delta_{t-1} = -1 $
- $ \delta_t = -1 $
- $ c = 0.5 $

**Cálculo:**

$$
R_t = (-1) \cdot (-4) - 0.5 \cdot |-1 - (-1)| = 4 - 0 = \boxed{4}
$$

**Lucro de 4 unidades**, pois o preço caiu e o agente estava vendido.

---

### Exemplo 4: Mudança de posição com custo, sem retorno imediato

- $ z_t = 2 $
- $ \delta_{t-1} = 0 $
- $ \delta_t = 1 $
- $ c = 0.5 $

**Cálculo:**

$$
R_t = 0 \cdot 2 - 0.5 \cdot |1 - 0| = 0 - 0.5 = \boxed{-0.5}
$$

***Perda de 0.5 unidades**, referente apenas ao custo de entrada.

---

### Exemplo 5: Mudança de posição com lucro subsequente

#### No tempo $ t $:

- $ z_t = 5 $
- $ \delta_{t-1} = 0 $
- $ \delta_t = 1 $
- $ c = 0.5 $

**Cálculo:**

$$
R_t = 0 \cdot 5 - 0.5 \cdot |1 - 0| = 0 - 0.5 = \boxed{-0.5}
$$

#### No tempo $ t+1 $:

- $ z_{t+1} = 3 $
- $ \delta_t = 1 $
- $ \delta_{t+1} = 1 $

**Cálculo:**

$$
R_{t+1} = 1 \cdot 3 - 0.5 \cdot |1 - 1| = 3 - 0 = \boxed{3}
$$

**Resumo**: perdeu 0.5 na entrada, mas ganhou 3 depois → **Lucro líquido: 2.5**

---



A função apresentada anteriormente (Equação 1) representa a **função de valor** típica usada em frameworks de *Deep Reinforcement Learning* (DRL). Essa função avalia o **lucro instantâneo** $ R_t $ em cada ponto do tempo.

Com base nesses valores $ R_t $, é possível definir o valor acumulado ao longo de todo o período de treinamento como:

$$
\max_{\Theta} \ U_T\{R_1, R_2, \dots, R_T \ | \ \Theta\}
\tag{2}
$$

Onde:

- $ U_T\{\cdot\} $ é a função que **acumula os lucros/recompensas** entre os tempos $ t = 1 $ até $ T $
- $ \Theta $ é a política de negociação, ou seja, a estratégia usada para tomar decisões ao longo do tempo
- O objetivo do treinamento é encontrar a política $ \Theta $ que **maximiza** esse valor acumulado

#### Recompensa acumulada mais simples: Total Profit (TP)

A forma mais direta de acumular as recompensas é simplesmente somar os lucros em todos os períodos:

$$
U_T = \sum_{t=1}^{T} R_t
$$

Essa soma representa o **lucro total (Total Profit – TP)** obtido no período completo de treinamento.

#### Alternativas mais sofisticadas

Embora o lucro total seja a forma mais intuitiva de recompensa, é possível utilizar **funções de recompensa mais complexas**, como:

- Retorno ajustado pelo risco (por exemplo, **Sharpe Ratio**)
- Lucro médio com penalizações por drawdown ou volatilidade

Estas opções são mencionadas como possíveis objetivos de aprendizado por reforço, mas **os autores escolhem usar o TP (lucro total)** como função objetivo principal ao longo do artigo, por simplificar a explicação e facilitar a implementação.

> Essas abordagens alternativas serão discutidas mais detalhadamente na Seção V do artigo.

### Aprendizado Direto da Política

Com a função de recompensa já bem definida, o desafio agora é:  **como otimizar essa função de forma eficiente**?

---

#### Limitações do Aprendizado Convencional por Reforço

Nos métodos clássicos de *Reinforcement Learning*, as funções de valor são definidas em **espaços discretos de estados** e otimizadas via **programação dinâmica** (como Q-Learning ou métodos de política-valor).

No entanto, os autores destacam que essa abordagem **não funciona bem para negociação financeira**, porque:

- O mercado é altamente **complexo e contínuo**
- É difícil representar as condições de mercado com um número razoável de **estados discretos**
- O ambiente é **não estacionário** e com ruídos, o que dificulta aprender o valor de cada estado

---

#### A Contribuição do DRL

Diante dessas limitações, a proposta de {cite:p}`Moody01` e aprofundada no artigo é:

> Em vez de aprender o valor de cada estado, **aprender diretamente a política** de decisão.

Esse paradigma é chamado de **Aprendizado por Reforço Direto** (*Direct Reinforcement Learning – DRL*).

---

#### Função da Política Direta 

A política de negociação é representada por uma **função não linear**, que estima diretamente a ação a ser tomada no tempo $ t $:

$$
\delta_t = \tanh\left( \langle \mathbf{w}, \mathbf{f}_t \rangle + b + u \cdot \delta_{t-1} \right)
\tag{3}
$$

Onde:

- $ \delta_t \in (-1, 1) $: saída contínua da política, representando a intensidade da posição (tanh aproxima os sinais $-1$, $0$, $1$)
- $ \mathbf{f}_t $: vetor de **features do mercado** no tempo $ t $ (ex: indicadores, retornos, volatilidade etc.)
- $ \mathbf{w} $: vetor de **pesos (parâmetros)** aprendidos para cada feature
- $ b $: **viés (bias)** da função
- $ u \cdot \delta_{t-1} $: termo de **memória**, que introduz dependência da ação anterior
- $ \langle \mathbf{w}, \mathbf{f}_t \rangle $: **produto interno** (ou somatório ponderado das features)

A função tangente hiperbólica $ \tanh(\cdot) $ comprime a saída para o intervalo $ (-1, 1) $, permitindo representar tanto posições compradas, neutras ou vendidas **de forma contínua**.

---

#### Interpretação
 
O modelo base de decisão do paper é uma rede neural extremamente simples, mas poderosa. Ela é composta por três partes principais:

---

1. Camada Linear

A primeira etapa da rede é uma **soma ponderada das entradas do mercado**, também chamada de produto interno.

- Cada feature do vetor de entrada tem um peso associado.
- Também há um termo de viés (bias), que serve como um deslocamento.
- Essa combinação resulta em um valor escalar que representa a **situação atual do mercado** conforme interpretada pela rede.

**Intuição:**  
É como um trader que diz:  
“Dou mais importância aos retornos recentes do que aos antigos, e tendo a favorecer posições compradas em geral.”

---

2. Feedback da Ação Anterior

O modelo incorpora um **termo de memória**, que é a **posição de trading tomada no tempo anterior multiplicada por um peso**.

- Isso permite ao modelo **levar em conta sua decisão passada** ao decidir o que fazer agora.
- A ideia é evitar trocas desnecessárias de posição, o que reduz custos operacionais (como taxas e slippage).
- Também permite que o modelo **favoreça a continuidade das ações** quando o mercado não muda drasticamente.

**Intuição:**  
É como um trader que pensa:  
“Se eu estava comprado ontem e não mudou muita coisa, talvez seja melhor continuar comprado.”

---

3. Ativação Tanh: Direção e Intensidade

Após a soma ponderada e o termo de memória, o resultado passa por uma **função de ativação tangente hiperbólica**.

- Essa função gera um valor contínuo entre -1 e 1.
- O resultado já é a **ação de trading**:
  - Valores próximos de -1 indicam posição vendida.
  - Valores próximos de 0 indicam posição neutra.
  - Valores próximos de 1 indicam posição comprada.
- Isso também permite **posições parciais**, como meio comprado ou meio vendido.

**Vantagem:**  
Não é necessário discretizar ações (como "comprar", "vender", "parar").  
O modelo já produz ações contínuas, mais suaves e mais fáceis de otimizar via gradiente.

---

Por que isso é elegante?

Com uma estrutura extremamente simples, o modelo já consegue:

- Ter **memória** do passado (recurrente).
- Fazer decisões **suaves e contínuas**.
- Aprender **diretamente dos dados brutos do mercado**.
- **Evitar regras manuais** ou discretização de estados/ações.

É uma forma poderosa e minimalista de tomar decisões em tempo real no mercado financeiro.




## Equação para $\bf f_t$  e a Estrutura do DRL

No modelo de **Reinforcement Learning Direto (DRL)**, os **m retornos mais recentes** são utilizados diretamente como vetor de entrada (features), representado por:
$$
{\bf f_t} = [z_{t-m+1}, ..., z_t] ∈ ℝ^m
$$


Além dessas features, um **termo adicional** $u * δ_{t-1}$ também é incluído na regressão. Esse termo representa a **decisão de trading do instante anterior** e é incorporado ao modelo para **desencorajar mudanças frequentes de posição**, com o objetivo de **evitar custos de transação elevados**.

A saída dessa soma linear é então passada por uma função de ativação **tangente hiperbólica (tanh)**, que mapeia o resultado para o intervalo **(-1, 1)**. Esse valor final é interpretado como a **decisão de trading do agente** naquele momento, podendo ser:

- Próximo de -1 → posição vendida
- Próximo de  0 → posição neutra
- Próximo de +1 → posição comprada

A **otimização do DRL** consiste em encontrar o melhor conjunto de parâmetros:
$$
Θ = {{\bf w}, u, b}
$$
que **maximiza a função de recompensa global** definida anteriormente.
